# Csci 5523 Final Project
## Rona Response Team

In [1]:
import numpy as np
import pandas as pd
import squarify
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree

In [2]:
raw_data = pd.read_csv('all-states-history.csv')
raw_data.head()
raw_data.columns

Index(['date', 'state', 'dataQualityGrade', 'death', 'deathConfirmed',
       'deathIncrease', 'deathProbable', 'hospitalized',
       'hospitalizedCumulative', 'hospitalizedCurrently',
       'hospitalizedIncrease', 'inIcuCumulative', 'inIcuCurrently', 'negative',
       'negativeIncrease', 'negativeTestsAntibody',
       'negativeTestsPeopleAntibody', 'negativeTestsViral',
       'onVentilatorCumulative', 'onVentilatorCurrently', 'positive',
       'positiveCasesViral', 'positiveIncrease', 'positiveScore',
       'positiveTestsAntibody', 'positiveTestsAntigen',
       'positiveTestsPeopleAntibody', 'positiveTestsPeopleAntigen',
       'positiveTestsViral', 'recovered', 'totalTestEncountersViral',
       'totalTestEncountersViralIncrease', 'totalTestResults',
       'totalTestResultsIncrease', 'totalTestsAntibody', 'totalTestsAntigen',
       'totalTestsPeopleAntibody', 'totalTestsPeopleAntigen',
       'totalTestsPeopleViral', 'totalTestsPeopleViralIncrease',
       'totalTestsViral'

In [3]:
columns_to_show = ['date','state','death','deathConfirmed', 'deathIncrease','hospitalizedCurrently','hospitalizedIncrease','negative','positive','positiveIncrease','totalTestResults','totalTestResultsIncrease']
data = raw_data[columns_to_show]
data.head()

,date,state,death,deathConfirmed,deathIncrease,hospitalizedCurrently,hospitalizedIncrease,negative,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease
0,2020-11-29,AK,121.0,121.0,0,159.0,1,975364.0,30816.0,612,1006180.0,7126
1,2020-11-29,AL,3577.0,3245.0,5,1609.0,0,1373770.0,247229.0,2236,1579713.0,5811
2,2020-11-29,AR,2470.0,2265.0,21,1030.0,24,1538738.0,156247.0,1221,1675828.0,10243
3,2020-11-29,AS,0.0,NaN,0,NaN,0,1988.0,0.0,0,1988.0,0
4,2020-11-29,AZ,6634.0,6148.0,10,2458.0,220,1920319.0,325995.0,3221,2236325.0,18441


In [4]:
MN_data = data[data['state']=='MN']
MN_data.head()

,date,state,death,deathConfirmed,deathIncrease,hospitalizedCurrently,hospitalizedIncrease,negative,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease
25,2020-11-29,MN,3578.0,3507.0,57,1785.0,220,2206087.0,312969.0,8946,4116741.0,89486
81,2020-11-28,MN,3521.0,3453.0,45,1785.0,380,2170530.0,304023.0,9022,4027255.0,105119
137,2020-11-27,MN,3476.0,3410.0,101,1785.0,277,2131220.0,295001.0,5698,3922136.0,49074
193,2020-11-26,MN,3375.0,3313.0,0,1812.0,0,2113887.0,289303.0,0,3873062.0,0
249,2020-11-25,MN,3375.0,3313.0,72,1812.0,322,2113887.0,289303.0,6387,3873062.0,63086


In [5]:
MN_data.to_csv('MN_Data.csv')

In [7]:
MN_data = pd.read_csv('MN_data.csv')
cols = MN_data.columns
MN_data = MN_data[cols[1:]]
MN_data.head()

,date,state,death,deathConfirmed,deathIncrease,hospitalizedCurrently,hospitalizedIncrease,negative,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease
0,2020-11-29,MN,3578.0,3507.0,57,1785.0,220,2206087.0,312969.0,8946,4116741.0,89486
1,2020-11-28,MN,3521.0,3453.0,45,1785.0,380,2170530.0,304023.0,9022,4027255.0,105119
2,2020-11-27,MN,3476.0,3410.0,101,1785.0,277,2131220.0,295001.0,5698,3922136.0,49074
3,2020-11-26,MN,3375.0,3313.0,0,1812.0,0,2113887.0,289303.0,0,3873062.0,0
4,2020-11-25,MN,3375.0,3313.0,72,1812.0,322,2113887.0,289303.0,6387,3873062.0,63086


In [8]:
MN_fall = MN_data[0:90]
MN_summer = MN_data[90:182]
MN_spring = MN_data[182:]

In [31]:
def discretize_data(arr, k):
    out = pd.DataFrame({'date': arr['date']})
    out['state'] = arr['state']
    cols = arr.columns[3:]
    for i in cols:
        bins = pd.cut(arr[i], k, 'retbins' == True, labels = list(range(k)))
        for j in range(k):
            out[i + ' bin'+str(j)] = np.zeros(len(arr))
            for row in arr.index:
                if bins.loc[row] == j:
                    out.loc[row, i + ' bin'+ str(j)] = 1
    return out      

In [36]:
new = discretize_data(MN_fall,4)

In [37]:
new.head()

,date,state,deathConfirmed bin0,deathConfirmed bin1,deathConfirmed bin2,deathConfirmed bin3,deathIncrease bin0,deathIncrease bin1,deathIncrease bin2,deathIncrease bin3,...,totalTestResults bin2,totalTestResults bin3,totalTestResultsIncrease bin0,totalTestResultsIncrease bin1,totalTestResultsIncrease bin2,totalTestResultsIncrease bin3,PosInc bins bin0,PosInc bins bin1,PosInc bins bin2,PosInc bins bin3
0,2020-11-29,MN,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2020-11-28,MN,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,2020-11-27,MN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2020-11-26,MN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2020-11-25,MN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
